<a href="https://colab.research.google.com/github/shivanialane/ML-Project/blob/main/gradioml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import gradio as gr

In [ ]:
# Load the data
train_url = "/content/train.csv"
test_url = "/content/test.csv"
train = pd.read_csv(train_url, parse_dates=['Dates'])
test = pd.read_csv(test_url, parse_dates=['Dates'])

In [ ]:
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [ ]:

# One-hot encode the categorical features
train = pd.concat([train, pd.get_dummies(train['DayOfWeek'])], axis=1)
train = pd.concat([train, pd.get_dummies(train['PdDistrict'])], axis=1)

In [ ]:
# Define the features and target
target = 'Category'
features = train.columns.drop(['Dates', 'Category', 'Descript', 'Resolution', 'Address', 'X', 'Y', 'DayOfWeek', 'PdDistrict'])


In [ ]:
# Define and fit the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train[features], train[target])

RandomForestClassifier(random_state=42)

In [ ]:
# Define the prediction function
def predict_crime(date, day_of_week, police_department_district, longitude, latitude):
    input_dict = {'Dates': date, 'DayOfWeek': day_of_week, 'PdDistrict': police_department_district, 'X': longitude, 'Y': latitude}
    input_df = pd.DataFrame(input_dict, index=[0])
    input_df = pd.concat([input_df, pd.get_dummies(input_df['DayOfWeek'])], axis=1)
    input_df = pd.concat([input_df, pd.get_dummies(input_df['PdDistrict'])], axis=1)
    input_df = input_df.reindex(columns=features, fill_value=0)
    prediction = model.predict_proba(input_df)[0]
    return {model.classes_[i]: float(prediction[i]) for i in range(len(prediction))}


In [ ]:
# Define the interface
inputs = [
    gr.inputs.Textbox(label="Date and Time", default="2015-05-12 12:00"),
    gr.inputs.Dropdown(choices=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], label="Day of Week"),
    gr.inputs.Dropdown(choices=['BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN'], label="Police Department District"),
    gr.inputs.Number(label="Longitude", default=-122.4194),
    gr.inputs.Number(label="Latitude", default=37.7749),
]
output = gr.outputs.Label(num_top_classes=38)

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python

In [ ]:
# Launch the interface
interface = gr.Interface(fn=predict_crime, inputs=inputs, outputs=output, title="San Francisco Crime Predictor")
interface.launch(debug=True,share=True);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d91cc7588ee7e87658.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [ ]:
import joblib

In [ ]:
joblib.dump(model, "rfmodel.joblib")